# Iterables, Iterators, And Generators
In this notebook, we learn about iterables, iterators, and generators. The notebook contains executable examples of these three types of python objects. We begin with iterables and iterators, which represent the foundational constructs. Then we introduce generators, which make it easier to define iterators.

The python documentation contains a very succinct description of iterables, iterators, and generators. In the [iterator types](https://docs.python.org/3/library/stdtypes.html#iterator-types) section of the documentation, iterables are referred to as "containers". It is also worth looking up the definitions of iterables and iterators in the python documentation's [glossary](https://docs.python.org/3/glossary.html).

# Overview
An iterable represents a container (a collection of items) that is capable of being iterated over. Iteration occurs by assigning one element at a time from the container to a local variable using python's built-in `for x in y:` language construct. The actual execution of such an iteration is carried out by using an object called an iterator, which encapsulates the process of a one-time loop over the container's elements.

Lists, dicts, and numpy arrays are all iterables, because you can say `for x in my_list:`, or `for x in my_dict:`, or `for x in my_array:`, and python will establish a loop that automatically binds x to the successive elements of these containers. Such functionality is not limited to built-in types. Python allows one to extend the `for x in y:` language construct so that it works for any arbitrary object `y` you care to define. One does this by making `y` an iterable.

To make an instance an iterable, its class must define an `__iter__()` method. Whenever the python interpreter sees a for-loop of the form `for x in y:`, it implements the loop by sending `y` an `__iter__()` message, and it expects the method to return an object called an iterator, which is responsible for doing the actual work of returning the elements of the container `y`. These get bound one at a time to the local variable `x`. 

An iterator does its job by defining a `__next__()` method, which returns successive elements from the container it represents each time `__next__()` is called, raising a `StopIteration` exception when there are no more elements to return. In addition to defining the `__next()__` method, an iterator must _also_ define the `__iter__()` method, returning itself as the value. These two methods constitute the [iterator protocol](https://docs.python.org/3/library/stdtypes.html#iterator-types). Note that, once an iterator raises `StopIteration` in response to the `__next__()` message, signalling that there are no more elements left, it must always and forever raise `StopIteration` on future calls to `__next__()`. An iterator that does not do so is considered a broken implementation.

Below is a simple example of an iterable and an illustration of its usage. Take a look at the code, and then we'll discuss how it works.

In [1]:
class SimpleIterable(object):
    """
    A collection of the integers between 0 and self.num_elements exclusive. When iterated over,
    the elements are returned in descending order.
    """
    def __init__(self, num_elements):
        self._num_elements = num_elements

    # This method is called implicitly by the python interpreter when it sees a for-loop. The method
    # must return an iterator object. It returns an instance of SimpleIterator, defined below.
    def __iter__(self):
        return SimpleIterator(self._num_elements)


class SimpleIterator(object):
    """
    An iterator over the integers num_elements (exclusive) down to 0.
    """
    def __init__(self, num_elements):
        self._last_element = num_elements # Keeps track of the previous element that was returned in the iteration.
        
    # This method is required of all iterators, and they must always return themselves.
    def __iter__(self):
        return self

    # This method is called by the python interpreter to get the next element from the iterator as part
    # of a for-loop. The method returns the next element in the container, keeping track of where it is 
    # in the iteration, and raises a StopIteration exception when it is all done.
    def __next__(self):
        self._last_element -= 1
        if self._last_element < 0:
            raise StopIteration
        return self._last_element

print("Running SimpleIterator Example")
container = SimpleIterable(5)  # Create a collection of the integers 0 through 4 inclusive.

# Loop through the successive elements in the container in descending order.
for element in container:
    print(element)

Running SimpleIterator Example
4
3
2
1
0


# Execution Details
Let's go through the details of how the for-loop above works. When the python interpreter sees the expression `for element in container:`, it expects container to be an iterable. The object `container` is an instance of class `SimpleIterable`, and it is indeed an iterable, by virtue of defining an `__iter__()` method. *Any* class that defines an `__iter__()` method causes its instances to be iterables. This is an example of "duck typing". The type is determined not by inheriting from a particular class, but just by virtue of  implementing a specified interface. In this case, the interface consists of a single method: the `__iter__()` method.

To implement the for-loop, the python interpreter begins by calling the iterable's `__iter__()` method and expects to receive back an iterator. In this case, it receives an instance of `SimpleIterator`. Just as container is an iterable by virtue of defining an `__iter__()` method, the `SimpleIterator` instance this method returns is an iterator by virtue of defining both an `__iter__()` method and a `__next__()` method, again by duck typing. The purpose of the `__next__()` method is to return successive elements of the original container each time `__next__()` is called.

The python interpreter then implements the for-loop by repeatedly calling the iterator's `__next__()` method and binding its returned values to the variable `element`. It expects the iterator to raise a `StopIteration` exception when there are no elements left to return, which occurrs just after it has returned the element 0. Python arranges to automatically trap this exception and exits from the loop when it occurs.

# Summary
An iterable is a container that knows how to loop through its elements by returning an iterator to do the job. Python establishes the interfaces that iterables and iterators must respect. Any object is an iterable if it defines an `__iter__()` method. Any object is an iterator if it defines both an `__iter__()` method that returns `self` and a `__next__()` method that returns successive elements from its associated container, raising a StopIteration exception after returning its last element.


# Built-in Functions Related to Iterators and Generators
There are some built-in functions in python you should know about that relate to iterables and iterators, allowing you to deal with them explicitly outside of the context of a for-loop. Even though 90% of the time you'll just invoke them implicitly via a for-loop, there are definitely times when it is desirable to have more control over the iteration, and this is the primary advantage one gains by using iterators.

## The iter() function
The `iter()` function is the pythonic way to call the `__iter__()` method of an iterable. As such, it returns an iterator over the iterable. For example, `iter(my_list)` returns an iterator over the elements of `my_list`. Similarly `iter(my_dict)` returns an iterator over the keys in `my_dict`. 

## The next() function
In like manner, the `next()` function is the pythonic way to call the `__next__()` method of an iterator. So, if `my_iterator` is an iterator, then `next(my_iterator)` returns the next element from its associated container. 

## The list() function
If `my_iterator` is an iterator, you can turn it into a list by calling `list()` on it. Note that this exhausts
the iterator, i.e.,  `my_iterator` will be empty after `list()` has been called on it. That is because `list()` does its job by calling `__next__()` on `my_iterator` to get all its elements.

## Examples of built-ins for iteration
Below are some examples illustrating these builtin functions.

In [2]:
print("Running examples illustrating built-in functions on iterators")
x = SimpleIterable(10) # The collection of integers from 0 through 9 inclusive.
y = iter(x)  # An iterator over the collection.

# The iterator gets depleted as next() is called on it, returning its elements one at a time.
print(next(y))  # -> 9
print(next(y))  # -> 8
print(next(y))  # -> 7

# prints 6,5,4
print()
for element in y:
    print(element)
    if element < 5:
        break
        
print()

# Once list() is called on my_iterator, the iterator becomes empty and stays empty.
print(list(y))  # -> [3,2,1,0]
print(list(y))  # -> []
try:
    next(y)  # -> raises StopIteration exception. 
             # my_iterator had all its remaining elements exhausted when list() was first called on it.
except StopIteration:
    print("y is now empty!")
    
    
print()
# By contrast, x is an iterable, not an iterator. The list() function can be called on x multiple times
# yielding the full list each time, because x returns a new iterator over its elements each time.
print(list(x))            # -> [9,8,7,6,5,4,3,2,1,0]   
print(list(x))            # -> [9,8,7,6,5,4,3,2,1,0]   

Running examples illustrating built-in functions on iterators
9
8
7

6
5
4

[3, 2, 1, 0]
[]
y is now empty!

[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]
[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]


Note that if you decide to handle iterators yourself by calling `next()` explicitly, then you also have to handle the `StopIteration` exception yourself by setting up the appropriate `try/except` block.

# Gotchas
There are lots of ways a for-loop over a custom object can go wrong, if you don't implement the iterator protocol properly. Here are some of them. If you write the expression `for x in my_obj:`, but `my_obj` does not define an `__iter__()` method, then this will crash with an error message saying that `__iter__()` is undefined for `my_obj`.

If the `__iter__()` method exists on `my_obj`, but the object it returns does not have a `__next__()` method, then this will crash with an error message saying that `__next__()` is not defined for the returned object. If the returned object never raises a `StopIteration` exception, then the for-loop will never terminate, unless it contains an explicit return or break within its body. There are times when this is exactly the desired behavior, i.e., when the iterable being looped over is intended to represent an infinite collection.

If the iterator returned by `my_obj` does not define an `__iter__()` method, or does not return itself as the value of that method, then this will crash if you invoke a for-loop on the remaining elements of the iterator.

# The Difference Between Iterables And Iterators
It might seem as though iterables and iterators are too close together in purpose to require they be distinct kinds of objects. In fact, there is an important distinction. 

Before getting to the distinction, let's first note that all iterators _are_ iterables, by duck-typing. An iterator is a special kind of iterable, which we might call a _one shot_ iterable. Inasmuch as it represents a container of elements, the elements an iterator represents keep changing. Each time `__next__()` is called, the iterator has one less element than it had before. Once the iterator is empty, it remains forever empty, hence the appellation _one shot_.

By contrast, an iterable need not be a one shot container. To the contrary, it is usually desirable for the set of elements an iterable represents _not_ to change. You can see this distinction in the previous code example above. On lines 32 and 33, the iterable x remains unchanged. Its contents never get depleted. By contrast, on lines 20 and 21, the iterator y is depleted, becoming empty after line 21.

By using both an iterable class and a separate iterator class, we can allow the iterable to represent an unchanging container that is *capable of being iterated over*, while the iterator represents a single iteration through the iterable's elements. If this distinction is not required for our use case, then we can get away with just defining an iterator class and calling it a day.

# The Difference Between Iterables And Containers
In practice there is very little difference between a container and an iterable. Certainly all iterables are containers. Conversely, it is hard to imagine a useful data-structure representing a container that does not also supply the ability to iterate through its contents. One counter-example might be an object representing all real numbers between `a` and `b`. Such an object can easily say whether or not it contains a given floating point number `x`, but iterating through all the floating point numbers between `a` and `b`, while possible, is unlikely to be useful. Whether such an object deserves to be called a _container_ is also a question of semantics.

We won't discuss the subject further. In practice, when we call an object an _iterable_, we wish to emphasize the fact that it can be iterated over, and when we call the same object a _container_, we wish to emphasize the fact that it is a collection of elements, probably providing several means of element-access beyond mere iteration, such as via position, in the case of a list, or via keyword in the case of a dict.


# Advantages of Custom Iterables
One of the important advantages of creating a custom iterable, in preference to merely using a list, is *lazy calculation*. In order to instantiate a list, the list must calculate all its elements first and store them all in memory. If the list is a long one, this can be intensive in both time and space. By contrast, an iterator only ever need allocate enough memory to calculate the next element in the iteration, and it only ever need expend enough computational effort to calculate that next element. It doesn't waste time calculating all the elements at once. In the `SimpleIterable` example above, suppose we had set `num_elements` to a large number such as one billion. This doesn't allocate any more memory than if we set `num_elements` to a small number such as 5, because the collection is represented abstractly and the iterator only ever calculates the next element from the previous element. By contrast, representing a list of one billion elements would take a few gigabytes.

Going further, what if the collection we want to represent is infinite? Then it would be impossible to create a list containing all its elements. Yet there are use cases for infinite collections. For example, the number of primes is infinite.  If we want to loop through all the primes while searching for the first one that satisfies some property, the most elegant way to do this is just to create an iterable that represents all primes.

Another advantage of iterables is that they give us a way to encapsulate filtering logic, making it separate from the client code that uses that logic. For example, the set of primes can be thought of as what one gets when one filters the set of positive integers for just those that are prime. Using an iterable gives us a way to wrap up the filtering logic, making it distinct from the client code that uses it. Client code can then ignore all the logic that goes into doing the filtering and concentrate instead on just its particular use case for primes. This makes client code simpler. Further, by separating out the filtering process from the client's use case, the iterable becomes re-usable by other clients that want to do something different with the container of primes. You can see this clarity illustrated in the examples below.

Related to filtering is _data re-organization_. If one has an iterable whose elements are not ideally structured for the task at hand, a second iterable can be interposed that restructures them so that the task at hand can be expressed more clearly, again allowing client code to hide cumbersome details that don't really relate to the meat of the problem. For example, if one's application is interested in pairs of consecutive primes, a second iterable can be created on top of the first primes iterable that restructures the data to return pairs of consecutive primes. This is also illustrated in the examples below.

# More Examples Of Iterables
The examples below illustrate all of the above advantages of iterables.

## Find the first prime greater than 100 whose remainder is 1 when divided by 7

In [9]:
from examples.primes.prime_utils import is_prime

# This example doesn't use custom iterables. It is clunkier than the solutions beneath it, because the
# filtering logic for primes has to be made explicit *inside client code*. This also means
# the filtering logic can't be re-used by different client code (without copy coding).
def find_prime_clunkily():
    """
    :return: the first prime greater than 100 equal to 1 mod 7.
    """
    x = 100
    while True:
        if is_prime(x):
            if x % 7 == 1:
                return x
        x += 1

print("Running find_prime_clunkily()")
print(find_prime_clunkily())


# This example uses a custom iterable that hides all the logic necessary to prepare the collection
# of primes greater than 100. The client-code can focus on its central purpose without getting bogged down
# by the details of filtering integers to just the primes.
def find_prime_elegantly():
    """
    :return: the first prime greater than 100 that is equal to 1 mod 7.
    """
    for prime in PrimesInRange(start = 100):
        if prime % 7 == 1:
            return prime


# Here we define the iterable and iterator objects necessary to allow the above elegant code to work.
# It's more work than the "clunky" example, but it has two advantages: it creates an iterable over primes
# that can be re-used by lots of different clients, and, in so doing, it hides all the logic so that
# client code will look cleaner.
class PrimesInRange(object):
    """
    Represents the collection of primes between integers self.start (inclusive) and self.finish (exclusive).
    If self.finish is not supplied at init(), it is taken to be infinity.
    """
    def __init__(self, start = 2, finish = float('inf')):
        """
        :param start: the lower bound (inclusive) for the sequence of primes to represent.
        :param finish: The upper bound (exclusive) for the sequence of primes to represent.
        """
        self._start  = start
        self._finish = finish
        
    @property
    def start(self):
        return self._start
    
    @property
    def finish(self):
        return self._finish

    def __iter__(self):
        """
        :return: an iterator over the primes represented by our collection.
        """
        return PrimesInRangeIterator(self)


class PrimesInRangeIterator(object):
    """
    Represents a single iteration through all primes between self.start (inclusive) and self.finish (exclusive)
    of our primes_in_range iterable.
    """
    def __init__(self, primes_in_range):
        """
        :param primes_in_range: A PrimesInRange iterable.
        """
        self._primes_in_range = primes_in_range
        self._next_to_test    = self._primes_in_range.start # Keeps track of the next integer we
                                                            # will test for primality.
            
    # Required by the iterator protocol.
    def __iter__(self):
        return self

    def __next__(self):
        """
        :return: the next prime in the specified range, or raise a StopIteration exception
                if we've already returned the last prime in range
        """
        found_prime = None
        while self._next_to_test < self._primes_in_range.finish:
            if is_prime(self._next_to_test):
                found_prime = self._next_to_test
                
            self._next_to_test += 1
            
            if found_prime:
                return found_prime
            
        raise StopIteration

print("Running find_prime_elegantly()")
print(find_prime_elegantly())


# The remaining examples just show some basic manipulation of the iterator derived from the initial iterable.
print()
print("Examples that manipulate the primes-iterable and its iterator")
x = PrimesInRange(finish = 100)
xx = iter(x)    # This is an iterator based on the iterable.
print(next(xx)) # -> 2
print(next(xx)) # -> 3

print()
print("Print the rest of the primes left in the iterator")
# Prints the remaining primes less than 100 from the iterator. 
for y in xx:
    print(y)

Running find_prime_clunkily()
113
Running find_prime_elegantly()
113

Examples that manipulate the primes-iterable and its iterator
2
3

Print the rest of the primes left in the iterator
5
7
11
13
17
19
23
29
31
37
41
43
47
53
59
61
67
71
73
79
83
89
97


# Find the first two consecutive primes greater than 100 whose difference is 8.
We now complicate matters a little bit. In this example our goal is to find the first two *consecutive* primes greater than 100 whose difference is precisely 8. The elegant way to do this is to build another iterable on top of the first. This second iterable will represent the sequence of consecutive prime pairs. The concept of building iterables on top of iterables is a powerful one that we will explore more in later sections.

In [11]:
# This example doesn't use custom iterables. It is clunkier because the
# filtering logic has to be made explicit *inside client code*
def find_primes_clunkily():
    """
    :return:  the first two successive primes greater than 100 whose difference is precisely 8.
    """
    # Initialize x1 to the first prime greater than or equal to 100
    x1 = 100
    while not is_prime(x1):
        x1 += 1

    while True:
        # Initialize x2 to the next prime strictly greater than x1
        x2 = x1 + 1
        while not is_prime(x2):
            x2 += 1
        
        # x1 and x2 now represent a pair of consecutive primes.
        # if they satisfy the desired constraint, we're done.
        if x2 - x1 == 8:
            return x1, x2
        
        # The value of x1 in the next prime pair will be the current value of x2.
        x1 = x2

print()
print()
print("Running find_primes_clunkily()")
print(find_primes_clunkily())

# This elegant implementation allows the client level logic to focus just on its central purpose, and hides all
# the nitty-gritty details of filtering integers to prime pairs.
def find_primes_elegantly():
    """
    Find the first two successive primes greater than 100 whose difference is precisely 8.
    :return:
    """
    for prime1, prime2 in ConsecutivePrimePairs(PrimesInRange(start = 100)):
        if prime2 - prime1 == 8:
            return prime1, prime2


class ConsecutivePrimePairs(object):
    """
    Represents the sequence of consecutive prime pairs from start (inclusive) to finish (exclusive).
    The sequence is conceived of like this: (2,3), (3, 5), (5, 7), (7, 11), ...
    """

    def __init__(self, primes_in_range):
        """
        :param primes_in_range: an iterable over the desired range of primes.
        """
        self._primes_in_range = primes_in_range  

    def __iter__(self):
        """
        Return an iterator over prime pairs in our range.
        """
        return ConsecutivePrimePairIterator(self._primes_in_range)


class ConsecutivePrimePairIterator(object):
    """
    Represents an iterator over the prime pairs implied by our primes_in_range iterable.
    """
    def __init__(self, primes_in_range):
        """
        :param primes_in_range: An iterable over the desired range of primes
        """

        self._primes_in_range = iter(primes_in_range)
        
        # It's conceivable the iterable is empty, so we carefully handle
        # grabbing its first element, in case it doesn't have one!
        try:
            self._last_prime = next(self._primes_in_range)  # Represents the last prime returned by the iterator.
        except StopIteration:
            self._last_prime = None

    def __next__(self):
        """
        :return: the next successive prime pair
        """
        prime            = next(self._primes_in_range) # this is the second element in the pair
        result           = self._last_prime, prime     # create the pair we're going to return.
        self._last_prime = prime                       # This will be the first element of the next pair.
        
        return result

print()
print()
print("Running find_primes_elegantly()")
print(find_primes_elegantly())



Running find_primes_clunkily()
(359, 367)


Running find_primes_elegantly()
(359, 367)


# Final Thoughts About Iterables And Iterators
Here is a final thought about iterables and iterators. One way to think of them is that they formalize the concept of iteration by creating an object that represents the iteration--one that can be manipulated programmatically--and there is power inherent in this ability.

By contrast, when you create a normal loop in python, the code that represents the loop is *not* an object that can be manipulated. It can't be passed around to other methods, and you can't programmatically ask it to run its body just once. But you *can* do this with an iterable. Calling the `__next__()` method of an iterator is equivalent to asking it to run the body of a loop just once. You can pass this loop body around to other methods, each of which might ask it to run the loop body one or more times. This is very powerful control. Whenever you have a normal loop, you could consider turning it into an iterable instead. Most of the time, you won't need to do this, but sometimes you will find that doing so gives you extra flexibility that you can take advantage of. Here's a typical example: suppose you have a main loop that embodies the functioning of some complex system whose inner workings you would like to "instrument", which means you want to print out some status information about the system each time through its main loop. Rather than embedding your instrumentation code throughout the system code, you can embody the system code in an iterable and put the instrumentation code in *client* code that calls next() on the iterable, followed by the instrumentation code. This allows you to completely separate the system-level code from the instrumentation code. Here's an outline of what this would look like:

    class MySystem(object):
      def __init__(self):
        self.foo = 0 # member variable of interest whose value changes each time through the main loop.
        self.bar = 0 # member variable of interest whose value changes each time through the main loop.
        self.baz = 0 # member variable of interest whose value changes each time through the main loop.

      def __iter__(self):
        return self

      def __next__(self):
        if self.time_to_stop():
          raise StopIteration()

        <implement the body of our main loop>

        return self


    # Here we instrument the system by printing out relevant member variables.
    # Note that the mere existence of the for-loop causes the system code to run
    # until it is done. The body of this loop only takes care of the instrumentation, printing out relevant
    # information.
    def my_instrumented_client_code():
      for sys in MySystem():
        print(sys.foo)
        print(sys.bar)
        print(sys.baz)

# Generators
Despite the elegance and flexibility achieved by using iterables, we must admit that, up till now, they have seemed rather cumbersome to implement. Up till now, we've needed to define at least one class, possibly two, and we've needed to define two methods: `__iter__()`, and `__next__()`, with the latter method usually being a little bit complicated, due to the need to preserve state between calls. Generator iterators unburden us from having to preserve state: they *automatically* cache *all* the state of a computation, and they do this for us implicitly, so that we don't have to worry about it. The language construct that allows this to happen is the `yield` statement. When `yield` is encountered in a function or method, all the internal state of the computation is *saved* so that the computation can be continued at a later time *exactly where it left off*. 

Put another way, _a generator iterator is an object that represents a frozen state of computation_. Generator iterators are implemented as iterators (as their name implies), which is to say that a generator iterator object answers to both the `__iter__()` and `__next_()` messages. In response to `__iter__()`, a generator iterator simply returns itself (as required by the iterator protocol). In response to `__next()__`, a generator iterator resumes execution from where it last left off until one of two things happens: either it encounters a `yield` statement, in which case it returns the arguments to `yield` as the values of `__next__()` and saves its computation state, or it encounters a `return` expression or "falls off" the end of its function body, in which case it raises a `StopIteration` exception.

It's important to emphasize that "functions" containing a `yield` statement always return immediately when they are "called", and the object they return is a _generator iterator_. This object represents a frozen state of computation. The way to tell the generator iterator to resume execution is to send it a `__next__()` message. This causes the generator iterator to continue execution just until it finishes execution of the next `yield` statement (or terminates by raising a StopIteration exception). The arguments to `yield` become the value(s) returned by `__next()__`.

We must pause to acknowledge that the term _generator iterator_ is a mouthful. Why not just call such an object a _generator_? If you re-read the previous paragraph above you'll notice we said that functions containing a `yield` statement return immediately, and the object they return is called a generator iterator. It is the function or method itself that is referred to as a _generator_. Thus, a generator is a function or method that, when called, returns a generator iterator. The distinction is usually clear from context, and, in practice, generator iterators are often referred to simply as generators. We will adopt this shorter, slightly ambiguous form for much of the rest of this notebook and will often refer to generator iterators simply as _generators_.

## Simple Generator Example
Below is a simple example of a generator. Take a look at it and then we'll discuss in detail how it works.

In [12]:
# The generator below is a replacement for the PrimesInRange iterable class that we encountered above.
# Note that it does its job without needing the PrimesInRangeIterable class either. 
# What used to take about 40 lines of code now takes 5. It's as short as find_primes_clunkily(), 
# which did not separate client code from filter code, but this version just does the filtering,
# so it is reusable by different clients.
def generate_primes_in_range(start, finish = float('inf')):
    """
    Return an iterator that yields primes in the range of start (inclusive) to finish (exclusive).
    :param start: the lower bound (inclusive) for the sequence of primes to represent.
    :param finish: The upper bound (exclusive) for the sequence of primes to represent.
    """
    x = start
    while x < finish:
        if is_prime(x):
            yield x
        x += 1

print("Running simple generator example")

# In the line below, we create a generator iterator of the primes between 4 and 15.
# It's important to realize that the function "call" to the generator generate_primes_in_range()
# returns *immediately*, with a generator iterator as its return value. 
# The body of the function has not yet begun executing. To make
# that happen, we need to send a __next__() message to primes_generator.
primes_generator = generate_primes_in_range(4, 15)  
                   

# Loop through the primes between 4 and 15, printing them.
# The python interpreter implicitly sends __next__() messages to primes_generator to cause it to execute
# and yield its elements.
for prime in primes_generator:
    print(prime)
    

Running simple generator example
5
7
11
13


When `generate_primes_in_range()` is called above on line 25, the python interpreter notices that the function contains a `yield` statement within its function body on line 15. The presence of the `yield` statement tells the interpreter to treat the function as a generator. The interpreter therefore handles the function call specially. It evaluates the arguments to the function and assigns their values to the function's parameters. It then suspends execution of the function and immediately returns without executing any of the code that is in its body. The object that it returns is called a _generator iterator_ object, and this is assigned to be the value of the variable `primes_generator`. This generator iterator object represents the state of execution of the function. As mentioned above, it is an iterator. It returns itself in response to the `__iter__()` message, and, in response to the `__next__()` message, it resumes execution from where it last left off.

Let's see how this plays out in line 31 above. The python interpreter notices the for-loop and therefore treats `primes_generator` as an iterable. It asks the iterable to supply an iterator for itself by calling `__iter__()` on it, and `primes_generator`, being an iterator, just returns itself.

Python then asks this iterator to return its next element by sending it a `__next__()` message. This resumes execution of the `generate_primes_in_range()` function from the point where it last left off, which was just assigning values to its parameters. The initial value of `x` is 4, but 4 is not prime, so the `yield` statement on line 15 is not evaluated. The next time through the loop, the value of `x` is 5. This *is* prime, so the `yield` statement is reached. This causes suspension of the execution of the function and the number 5 is returned as the value of the `__next__()` method.

The python interpreter, in executing the for-loop, continues sending `__next__()` messages to the generator iterator. Every time the generator iterator is sent a `__next__()` message, it resumes execution where it last left off, and keeps executing until one of two things happens: either it evaluates a `yield` statement, in which case the argument(s) to yield are returned as the value(s) of `__next__()`, or it evaluates a return expression (which includes "falling off" the end of the function), in which case, instead of returning from `__next__()`, it raises a `StopIteration` exception. In the example above, there is no explicit return statement in the function
body. However, when `x` hits the value of `finish` (which is set to 15), the while loop exits and the function "falls off" the end of its code body, which implicitly results in a return and raises a `StopIteration` exception that causes the interpreter to terminate the for-loop. 

# Redoing the prime number examples with generators
The prime number examples above were used to illustrate the fundamental mechanics of iterables and iterators. Now that we know about generators, the examples can be re-written in fewer lines of code, while maintaining their flexibility, clarity, and separation of client-logic from filtering logic. Some of the examples are re-done more than once, to illustrate that we can solve these problems with generators alone, but also by combining generators with classes that represent iterables. Although the latter technique requires more code, it also has some advantages, because an interable can be interrogated about its state in a way that a generator cannot.


## Find the first prime greater than 100 whose remainder is 1 when divided by 7--take 2
Below we re-implement the task of finding the first prime greater than 100 that is equal to 1 mod 7. We re-use the generator `generate_primes_in_range()`, defined in the example above. Note that this re-implementation uses no classes, so it is quite compact, comprising 7 lines of client code and about 10 lines for the generator defined above.

In [13]:
def find_prime_elegantly2():
    """
    :return: the first prime greater than 100 that is equal to 1 mod 7.
    """
    for prime in generate_primes_in_range(start = 100):
        if prime % 7 == 1:
            return prime
        
print("Running find_prime_elegantly2()")
print(find_prime_elegantly2())

Running find_prime_elegantly2()
113


## Find the first prime greater than 100 whose remainder is 1 when divided by 7--take 3
Below we again re-implement the task of finding the first prime greater than 100 that is equal to 1 mod 7. This time we create an iterable class, but not an iterator class. This architecture is quite flexible, because the iterable can be interrogated by client code for things like its "start" and "finish" values, in a way that the generator version above cannot be. But using a generator to implement the iterator makes life a lot simpler.

In [14]:
from examples.primes.prime_utils import is_prime

def find_prime_elegantly3():
    """
    :return: the first prime greater than 100 that is equal to 1 mod 7.
    """
    for prime in PrimesInRange2(start = 100):
        if prime % 7 == 1:
            return prime


# Here we define an iterable that represents the set of primes in a given range. We
# use a generator to implement the iterator that needs to be returned by __iter__().
class PrimesInRange2(object):
    """
    Represents the collection of primes between integers self.start (inclusive) and self.finish (exclusive).
    If self.finish is not supplied at init(), it is taken to be infinity.
    """
    def __init__(self, start = 2, finish = float('inf')):
        """
        :param start: the lower bound (inclusive) for the sequence of primes to represent.
        :param finish: The upper bound (exclusive) for the sequence of primes to represent.
        """
        self._start = start
        self._finish = finish
        
    @property
    def start(self):
        return self._start
    
    @property
    def finish(self):
        return self._finish

    # Here we just put the same code that was used in generate_primes_in_range(). By embedding the iterator
    # inside this class we can use instances of the class to represent the primes in a given range, and such
    # instances can be interrogated about themselves. For example, self.start and self.finish tell the client 
    # about the range in question.  The generator-only example doesn't have this ability.
    def __iter__(self):
        """
        :return: an iterator over the primes represented by our collection.
        """
        x = self.start
        while x < self.finish:
            if is_prime(x):
                yield x
            x += 1

print("Running find_prime_elegantly3()")
print(find_prime_elegantly3())

Running find_prime_elegantly3()
113


# Find the first two consecutive primes above 100 whose difference is 8--take 2
Here we re-implement the consecutive prime pairs example. We choose to re-implement it by retaining the ConsecutivePrimePairs iterable class, and just define its `__iter__()` method using a generator. As in the previous example, we could have also re-implemented it without any classes at all; however, having already illustrated that architecture above, we don't repeat it here. For all but the simplest cases, it probably makes more sense to define a class to represent the container in question.

In [16]:
def find_primes_elegantly2():
    """
    Find the first two successive primes greater than 100 whose difference is precisely 8.
    :return:
    """
    for prime1, prime2 in ConsecutivePrimePairs2(PrimesInRange(start = 100)):
        if prime2 - prime1 == 8:
            return prime1, prime2


class ConsecutivePrimePairs2(object):
    """
    Represents the sequence of consecutive prime pairs from start (inclusive) to finish (exclusive).
    The sequence is conceived of like this: (2,3), (3, 5), (5, 7), (7, 11), ...
    """

    def __init__(self, primes_in_range):
        """
        :param primes_in_range: an iterable over the desired range of primes.
        """
        self._primes_in_range = primes_in_range  

    def __iter__(self):
        """
        Return an iterator over prime pairs in our range.
        """
        primes_in_range = iter(self._primes_in_range) # Turn iterable into iterator
        last_prime      = next(primes_in_range)
        prime           = next(primes_in_range)
        while True:
            yield last_prime, prime
            
            last_prime = prime
            prime      = next(primes_in_range)

print("Running find_primes_elegantly()")
print(find_primes_elegantly())

Running find_primes_elegantly()
(359, 367)


# Iterators Can Be Chained
The consecutive prime pairs examples above illustrate how iterables can be built on top of each other, with the output of one iterator feeding the input of the next. Such a data architecture is referred to as _chained_. Iterators make it very easy to chain data, for the purposes of data reorganization or filtering. In this final section, we emphasize the point by re-implementing each of the two examples involving primes in such a way as to add one more layer of chaining. 

The examples here are not particularly well-motivated. In practice, it is hard to imagine why creating iterables that represent primes equal to 1 mod 7, or creating iterables that represent consecutive prime pairs separated by 8, would be generically reusable. The point to be made is that the _technique_ illustrated is useful. If the set of primes equal to 1 mod 7 _were_ generally useful, then the way to codify that collection is with an iterable.

## Find the first prime greater than 100 whose remainder is 1 when divided by 7--take 4


In [18]:
# There is an infinite set of interest, namely: those primes greater than 100 whose remainder is 1 when divided by 7. 
# We can represent this set with an iterator built on top of PrimesInRange2. In the special case at hand,
# we only want the first such prime, so we can ask for it by using next(), as is done in
# the code below. Note also that we leverage the iterable PrimesInRange2 that was defined previously above.
def find_prime_elegantly4():
    """
    :return: the first prime greater than 100 that is equal to 1 mod 7.
    """
    return next(generate_x_mod_y(PrimesInRange2(start = 100), modulus = 7, remainder = 1))
            
# Here we introduce a new iterator that filters its input to just those integers equal to remainder mod modulus.
def generate_x_mod_y(container, modulus, remainder):
    """
    :return: a generator iterator representing the subset of integers from container
             whose remainder is <remainder> when divided by <modulus>.
    
    :param container: an iterable representing a container of integers
    :param remainder: We will filter container to those integers that are equal to this remainder mod modulus.
    :param modulus: We will filter container to those integers are equal to this remainder mod modulus.
    """ 
    for x in container:
        if x % modulus == remainder:
            yield x
            
print("Running find_prime_elegantly4()")
print(find_prime_elegantly4())


Running find_prime_elegantly4()
113


## Find the first two consecutive primes above 100 whose difference is 8--take 3

In [21]:
# As in the re-implementation above, we create a new layer of iterable that represents a filter on
# pairs of integers, yielding just those pairs separated by 8.
# Then we simply return the first element from the iterable.
def find_primes_elegantly3():
    return next(generate_constant_difference_pairs(ConsecutivePrimePairs2(PrimesInRange(start = 100)), 8))

def generate_constant_difference_pairs(pairs_container, difference):
    """
    :return: an iterator over just those pairs in pairs_container separated by <difference>.
    
    :param pairs_container: an iterable representing pairs of integers
    """
    for x1, x2 in pairs_container:
        if x2 - x1 == difference:
            yield x1, x2
        
print("Running find_primes_elegantly3()")
print(find_primes_elegantly3())


Running find_primes_elegantly3()
(359, 367)
